In [ ]:
from Bio import Entrez
from Bio import SeqIO

def find_latest_genome(email, species, genome_type):
    # Define the search term for the species and genome type
    search_term = f"{species}[Organism] AND {genome_type}[Title]"

    # Use the Entrez API to search for genomes
    Entrez.email = email
    handle = Entrez.esearch(db="nucleotide", term=search_term, sort="most recent")
    record = Entrez.read(handle)
    handle.close()

    # Get the list of matching genome IDs
    genome_ids = record["IdList"]

    if not genome_ids:
        return None  # No genomes found

    # Get the most recent genome record
    latest_genome_id = genome_ids[0]

    # Retrieve the genome record
    handle = Entrez.efetch(db="nucleotide", id=latest_genome_id, rettype="gb", retmode="text")
    genome_record = SeqIO.read(handle, "genbank")
    handle.close()

    return genome_record

def get_reference_sequence(genome_record):
    # Determine the ideal reference sequence (usually the primary assembly)
    for feature in genome_record.features:
        if feature.type == "source" and "refseq" in feature.qualifiers:
            return feature.qualifiers["refseq"][0]

    # If no ideal reference sequence is found, return the first sequence in the record
    return genome_record.id

def is_refseq_available(genome_record):
    # Check if the "refseq" qualifier is present in the features
    for feature in genome_record.features:
        if feature.type == "source" and "refseq" in feature.qualifiers:
            return True
    return False

if __name__ == "__main__":
    # Prompt the user for their email
    email = input("Please enter your email: ")

    # Enter a single genome type for all species
    genome_type = input("Please enter the type of genome (e.g., 'whole' or 'partial'): ")

    # Enter multiple species
    species_list = input("Please enter the species in 'Genus species' format, separated by commas: ").split(',')

    for species in species_list:
        latest_genome = find_latest_genome(email, species, genome_type)

        if latest_genome:
            reference_sequence = get_reference_sequence(latest_genome)
            publication_date = latest_genome.annotations.get("date", "Date not available")
            refseq_available = is_refseq_available(latest_genome)

            # Print the information for each species in columns
            print(f"Species: {species}")
            print(f"Genome Type: {genome_type}")
            print(f"Description: {latest_genome.description}")
            print(f"Reference Sequence: {reference_sequence}")
            print(f"Publication Date: {publication_date}")
            print(f"RefSeq Available: {refseq_available}")
            print("\n")
        else:
            # Print a message for species with no genomes found
            print(f"No genome found for {species}\n")

